# Investigation of alternative to baseline-dependent averaging.

The idea is to partition the visibility into inner and outer parts. The density of samples in the inner parts of the uv plane is much greater so we can afford to use coarser gridding but with the same field of view as used for the
further out data. The computation required for the inner, dense region of the Fourier plane is just a complex addition, instead of a complete gridding kernel complex add. This approach should rival two-region baseline-dependent averaging.

In this notebook, we investigate the consequences of imaging in this way.

In [ ]:
%matplotlib inline

import os
import sys

from time import clock

sys.path.append(os.path.join('..', '..'))

from matplotlib import pylab

pylab.rcParams['agg.path.chunksize'] = 10000
pylab.rcParams['figure.figsize'] = (10.0, 10.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.convolution import Gaussian2DKernel, convolve
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy import constants as const
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib import pyplot as plt
from matplotlib.pyplot import cm 

from arl.visibility.operations import create_visibility
from arl.skymodel.operations import create_skycomponent, insert_skycomponent
from arl.image.operations import show_image, export_image_to_fits, qa_image, create_image_from_array, reproject_image
from arl.fourier_transforms.fft_support import extract_mid
from arl.image.iterators import raster_iter
from arl.visibility.iterators import vis_timeslice_iter
from arl.util.testing_support import create_named_configuration, create_low_test_image
from arl.fourier_transforms.ftprocessor import *
from arl.visibility.compress import compress_visibility, decompress_visibility

import logging
log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

Construct the SKA1-LOW configuration

In [ ]:
low = create_named_configuration('LOWBD2')

To see improvement, the sampling must be sufficiently fine on the longer baselines. Hence we simulate only a very limited house angle range.

In [ ]:
oversampling = 2
sampling_time = 35.0 / (oversampling * 8e4)
log.info("Critical sampling time = %.5f (radians) %.2f (seconds)" % 
         (sampling_time, sampling_time * 43200.0 / numpy.pi))
sampling_frequency = 1e8 * 35.0 / (oversampling * 8e4) 
log.info("Critical sampling frequency = %.5f (Hz) " % (sampling_frequency))
times = numpy.arange(- sampling_time, + sampling_time, sampling_time)
frequency = numpy.linspace(1e8 - sampling_frequency, 1e8 + sampling_frequency, 3)
print("Observing frequencies %s Hz" % (frequency))

We create the visibility holding the vis, uvw, time, antenna1, antenna2, weight columns in a table. The actual visibility values are zero.

In [ ]:
phasecentre = SkyCoord(ra=+180.0 * u.deg, dec=-60.0 * u.deg, frame='icrs', equinox=2000.0)
vt = create_visibility(low, times, frequency, weight=1.0, phasecentre=phasecentre, npol=1)

In [ ]:
plt.clf()
plt.plot(+vt.u, +vt.v, '.', color='r')
plt.plot(-vt.u, -vt.v, '.', color='r')
plt.show()

Create a model to serve as the image specification

In [ ]:
vt.data['vis'] *= 0.0
npixel=4096

model = create_low_test_image(npixel=npixel, cellsize=0.00001)

vt = predict_2d(vt, model)

In [ ]:
inners = [1.0/64.0, 1.0/32.0, 1.0/16.0, 1.0/8.0, 1.0/4.0]
times = []
qas = []
for boundary in inners:

    visr = numpy.sqrt(vt.u**2+vt.v**2)
    uvmax = numpy.max(visr)
    inner_rows = (numpy.abs(visr)  < boundary * uvmax) & (numpy.abs(visr) <  boundary * uvmax)
    inner_vis = create_visibility_from_rows(vt, inner_rows) 
    inner_im = create_image_from_visibility(inner_vis, npixel=2048, cellsize=0.00001/boundary,
                                           image_nchan=1)
    vts, density, densitygrid = weight_visibility(inner_vis, inner_im)
    plt.clf()
    uvdist = numpy.sqrt(vts.u*vts.u+vts.v*vts.v)
    plt.semilogy(uvdist[density[:,0,0]>0.0], density[:,0,0][density[:,0,0]>0], '.')
    plt.title('Density of visibility subset')
    plt.xlabel('UV distance (m)')
    plt.ylabel('Density (weight per cell)')
    plt.show()
    
    inner_vis_compressed = compress_visibility(inner_vis, inner_im)
    plt.clf()
    plt.title('UV coverage of compressed visibility')
    plt.plot(inner_vis_compressed.u, inner_vis_compressed.v, '.', color='r')
    plt.plot(-inner_vis_compressed.u, -inner_vis_compressed.v, '.', color='b')
    plt.xlabel('U (m)')
    plt.xlabel('V (m)')
    plt.show()

First use invert_timeslice to get the benchmark time.

In [ ]:
model = create_image_from_visibility(vt, npixel=npixel, cellsize=0.00001, npol=1, 
                                     image_nchan=1)

ts = clock()
full_dirty, full_sumwt = invert_2d(vt, model, dopsf=False, nprocessor=1)
full_dirty = normalize_sumwt(full_dirty, full_sumwt)

criterion = 'medianabs'
qa_full = qa_image(full_dirty).data[criterion]
t_full = clock() - ts
log.debug("Invert_2d took %.1f seconds" % (t_full))
export_image_to_fits(full_dirty, 'imaging-twopart_dirty_full.fits')

Now do a range of partitions

In [ ]:
inners = [1.0/64.0, 1.0/32.0, 1.0/16.0, 1.0/8.0, 1.0/4.0]
times = []
qas = []
for inner in inners:
    ts = clock()
    twopart_dirty, twopart_sumwt = invert_nested(vt, model, dopsf=False, invert=invert_2d, inner=inner,  
                                                 npixel=npixel, cellsize=0.00001,image_nchan=1)
    t_twopart = clock() - ts
    log.debug("invert_nested using inner=%.3f took %.1f seconds" % (inner, t_twopart))
    times.append(t_twopart)
    
    twopart_dirty = normalize_sumwt(twopart_dirty, twopart_sumwt)
    export_image_to_fits(twopart_dirty, 'imaging-twopart_dirty_full%.3f.fits' % inner)   

    twopart_dirty.data-=full_dirty.data

    thisqa = qa_image(twopart_dirty)
    print(thisqa)
    qas.append(thisqa.data[criterion]) 
    
    show_image(twopart_dirty)
    plt.title("Error in dirty for inner = %.3f * uvmax" % inner)
    plt.show()

In [ ]:
print(times)
print(qas)
plt.clf()
plt.plot(inners, times, label='twopart', color='g')
plt.axhline(t_full, label='2d', color='r')
plt.xlabel('Fraction of uvmax')
plt.ylabel('Time (s)')
plt.title('Processing time as function of fractional uvmax')
plt.ylim(0.0, 1.1*t_full)
plt.legend()
plt.show()
plt.clf()
plt.plot(inners, qas, label='twopart', color='g')
plt.axhline(qa_full, label='2d', color='r')
plt.xlabel('Fraction of uvmax')
plt.ylabel('Median abs')
plt.ylim(0.0, 1.1*qa_full)

plt.legend()
plt.title('Median absolute error as function of fractional uvmax')
plt.show()